In [1]:
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
import re
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard
from tensorflow.keras.optimizers.schedules import PolynomialDecay
from tensorflow.keras import regularizers

2024-02-09 09:43:45.399933: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-02-09 09:43:45.596160: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-09 09:43:45.596202: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-09 09:43:45.608003: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-09 09:43:45.631432: I tensorflow/core/platform/cpu_feature_guar

In [2]:
df = pd.read_csv("arnav.csv")

In [3]:
def preprocess(text):
    text = re.sub("[_\n]", " ", text)
    return re.sub(r" +", "", text)


df["text"] = df["text"].apply(preprocess)

In [4]:
bert_preprocess = hub.KerasLayer(
    "https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3"
)
bert_encoder = hub.KerasLayer(
    "https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/3"
)

2024-02-09 09:43:50.882830: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-02-09 09:43:50.988097: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-02-09 09:43:50.988259: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

In [5]:
label_encoder = LabelEncoder()
df["label"] = label_encoder.fit_transform(df["label"])

In [6]:
df = df.sample(frac=1, random_state=30)
train_data, temp_data = train_test_split(
    df, test_size=0.2, stratify=df["label"], random_state=30
)
dev_data, test_data = train_test_split(
    temp_data, test_size=0.5, stratify=temp_data["label"], random_state=30
)

In [7]:
import numpy as np

X_train = np.array((train_data["text"].tolist()))
y_train = np.array(to_categorical(train_data["label"].tolist(), num_classes=10))
X_dev = np.array((dev_data["text"].tolist()))
y_dev = np.array(to_categorical(dev_data["label"].tolist(), num_classes=10))
X_test = np.array((test_data["text"].tolist()))
y_test = np.array(to_categorical(test_data["label"].tolist(), num_classes=10))

In [8]:
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name="text")
preprocessed_text = bert_preprocess(text_input)
outputs = bert_encoder(preprocessed_text)

In [9]:
lambda_reg = 0.01

In [10]:
dropout_layer1 = tf.keras.layers.Dropout(0.1)(outputs["pooled_output"])
dense_layer1 = tf.keras.layers.Dense(
    512, activation="relu", kernel_regularizer=regularizers.l2(lambda_reg)
)(dropout_layer1)
dropout_layer2 = tf.keras.layers.Dropout(0.1)(dense_layer1)
dense_layer2 = tf.keras.layers.Dense(
    512, activation="relu", kernel_regularizer=regularizers.l2(lambda_reg)
)(dropout_layer2)
output = tf.keras.layers.Dense(10, activation="softmax", name="output")(dense_layer2)
model = tf.keras.Model(inputs=[text_input], outputs=output)

In [11]:
batch_size = 32

In [12]:
STEPS_PER_EPOCH = len(X_train) // batch_size
#
custom_lr = PolynomialDecay(
    initial_learning_rate=0.01, decay_steps=STEPS_PER_EPOCH, end_learning_rate=0.0001
)
# # Now you can use STEPS_PER_EPOCH in your learning rate schedule
# custom_lr = tf.keras.optimizers.schedules.InverseTimeDecay(
#     0.001, decay_steps=STEPS_PER_EPOCH * 1000, decay_rate=1, staircase=False
# )

In [13]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=custom_lr),
    loss="categorical_crossentropy",
    metrics=["accuracy"],
)

2024-02-09 09:44:59.434230: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory


In [14]:
early_stopping = EarlyStopping(monitor="val_loss", patience=3)
checkpoint = ModelCheckpoint(
    "best_model.h5", monitor="val_accuracy", verbose=1, save_best_only=True, mode="max"
)
tensorboard = TensorBoard(log_dir="./logs")

In [15]:
model.evaluate(X_train, y_train)

3/3 [==============================] - 2s 206ms/step - loss: 13.7298 - accuracy: 0.0875


[13.729833602905273, 0.08749999850988388]

In [16]:
model.evaluate(X_test, y_test)

1/1 [==============================] - 0s 100ms/step - loss: 13.7039 - accuracy: 0.0000e+00


[13.703880310058594, 0.0]

In [17]:
model.evaluate(X_dev, y_dev)

1/1 [==============================] - 0s 95ms/step - loss: 13.7007 - accuracy: 0.1000


[13.700742721557617, 0.10000000149011612]

In [18]:
history = model.fit(
    X_train,
    y_train,
    validation_data=(X_dev, y_dev),
    epochs=300,
    batch_size=batch_size,
    callbacks=[early_stopping],
)

Epoch 1/300


2024-02-09 09:45:05.830024: I external/local_xla/xla/service/service.cc:168] XLA service 0x7f6b9c0021e0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-02-09 09:45:05.830055: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 3060 Laptop GPU, Compute Capability 8.6
2024-02-09 09:45:05.835208: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-02-09 09:45:05.857695: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8904
I0000 00:00:1707452105.914996   11546 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


3/3 [==============================] - 5s 264ms/step - loss: 23.0035 - accuracy: 0.0750 - val_loss: 16.7646 - val_accuracy: 0.1000
Epoch 2/300
3/3 [==============================] - 1s 248ms/step - loss: 16.8629 - accuracy: 0.1125 - val_loss: 15.7772 - val_accuracy: 0.1000
Epoch 3/300
3/3 [==============================] - 1s 248ms/step - loss: 15.9484 - accuracy: 0.1500 - val_loss: 14.8642 - val_accuracy: 0.1000
Epoch 4/300
3/3 [==============================] - 1s 242ms/step - loss: 14.9894 - accuracy: 0.2000 - val_loss: 14.0975 - val_accuracy: 0.0000e+00
Epoch 5/300
3/3 [==============================] - 1s 246ms/step - loss: 14.1724 - accuracy: 0.1625 - val_loss: 13.4024 - val_accuracy: 0.0000e+00
Epoch 6/300
3/3 [==============================] - 1s 247ms/step - loss: 13.3334 - accuracy: 0.1875 - val_loss: 12.7819 - val_accuracy: 0.1000
Epoch 7/300
3/3 [==============================] - 1s 256ms/step - loss: 12.7754 - accuracy: 0.1750 - val_loss: 12.2644 - val_accuracy: 0.1000
Epo

In [20]:
model.evaluate(X_test, y_test)
y = model.predict(X_test)
predicted_indices = np.argmax(y, axis=-1)
original_labels = label_encoder.inverse_transform(predicted_indices)

1/1 [==============================] - 0s 28ms/step


In [21]:
print(original_labels)

['legal ' 'creative' 'business' 'government' 'scientific' 'government'
 'business' 'scientific' 'medical' 'medical']
